In [ ]:
#Resultados mais relevantes
#População e resultados
#Maiores e menores gastos (incluindo senadores que não utilizaram a CEAPS)
#Boxplot

# Introdução
<br>
<div style="text-align: justify"> 
   
- A Cota para Exercício da Atividade Parlamentar dos Senadores (CEAPS) é o valor a que tem direito os senadores para realização de atividades relacionadas ao seu mandato. A cota individual por senador é variável de acordo com a distância do estado do senador a Brasília. Esse valor é determinado pela Mesa Diretora do Senado, bem como as regras relativas. A utilização da cota se dá por meio de reembolso: o senador realiza a despesa e é reembolsado pelo Senado Federal. 
    
- Os valores da CEAPS são dinheiro público, portanto de interesse geral, tanto em quantidade quanto em qualidade do gasto. Qualquer cidadão pode acessar todos os gastos dos senadores relacionados à essa cota, inclusive verificando as imagens dos documentos que originaram esses reembolsos. Essa informação é publica no site do Senado e está amparada na Lei de Acesso à Informação. 
    
- A CEAPS é, dentro das regras estabelecidas, de uso discricionário do parlamentar. Ou seja, não existe necessidade de realização de pregão ou concorrência pública para contratação dos produtos e serviços. Isso torna ainda mais necessária a vigilância pública sobre seu uso a fim de detectar eventuais ilícitos e coibir práticas lesivas ao dinheiro do contribuinte. Da mesma forma, o parlamentar pode optar por não usar qualquer valor da cota, custeando as despesas do seu próprio salário. 

- **É importante destacar que a CEAPS não contempla todos os custos suportados pelos contribuintes gerados pelos senadores e suas atividades**. Não entram nesse montante despesas relevantes como pagamento de funcionários (incluindo efetivos, comissionados e terceirizados), combustível para uso de carro oficial, passagem e hospedagem em viagens oficiais, auxílio-moradia e o próprio salário do senador. Nesse caso, os custos são pelo Senado Federal, este por sua vez também dinheiro do contribuinte brasileiro.

#### Categorias da CEAPS
Os custos incorridos pelos senadores podem se encaixar em uma das categorias abaixo para que o parlamentar tenha direito ao reembolso.
1.	**Aluguel de imóveis para escritório político**: o chamado escritório político é o local, no estado do parlamentar, em que concentram suas atividades parlamentares. Nessa categoria estão os custos com aluguel, água, energia elétrica, telefonia e internet, bem como custos de condomínio e IPTU.
2.	**Aquisição de material de consumo**: demais despesas relacionadas ao escritório político que não se encaixam na anterior, como manutenção de computadores, correios e papel para impressão. As despesas do gabinete do senador em Brasília não entram aqui.
3.	**Passagens aéreas, aquáticas e terrestres nacionais**: compras de passagens para uso pelo senador ou servidor do seu gabinete ou escritório político exclusivamente do estado do senador para Brasília e vice-versa. 
4.	**Locomoção, hospedagem, alimentação e combustíveis**: gastos relacionados à locomoção do senador ou servidores, como serviços de táxi e locação de veículos, hospedagem, alimentação, como restaurantes, churrascarias e lanchonetes, e combustíveis. O combustível gasto em carros oficiais não está incluído aqui.
5.	**Contratação de serviços de apoio ao parlamentar**: serviços de consultoria, assessoria e pesquisas, como elaboração de pareceres jurídicos.
6.	**Divulgação da atividade parlamentar**: serviços de comunicação para divulgação da atividade do senador, como campanhas de publicidade, material impresso e inserções em rádio e TV.
7.	**Serviços de segurança privada**: serviços de segurança prestada por empresa terceirizada, como guarda-costas e escolta. Os gastos com a Polícia do Senado Federal não estão inclusos aqui. 
</div>



In [ ]:
import calendar
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
import requests
import seaborn as sns
import textwrap

%matplotlib inline

**Parâmetros importantes para `pd.read_csv`**
Ao importar arquivos CSV em português, a atenção a dois parâmetros pode ser importante.
- `encoding`: a codificação de arquivo mais comum usada em português é ***utf-8***, no entanto (como nesse caso) isso pode gerar erros na leitura. Nesse caso, é provável que seja utilizada ***latin_1***.
- `decimal`: em português, a vírgula é usada como separador decimal. Como o padrão do `pandas` é do inglês (em que a vírgula é separador de milhar), é conveniente especificar esse parâmetro quando o arquivo contém dados do tipo ponto flutuante/*float*.


In [ ]:
ceaps = pd.read_csv("./despesa_ceaps_2021.csv", sep=";", encoding="latin_1", header=1, decimal=",")
senadores = pd.read_csv("./senadores(2019-2023).csv", sep=";")
empresas = pd.read_csv("./dados_empresas_ceaps_2021.csv", encoding="utf-8")
populacao = pd.read_csv("./estados_populacao.csv", sep=";", thousands=".")

In [ ]:
ceaps.info()

In [ ]:
ceaps.head()

In [ ]:
ceaps["ANO"].unique()

In [ ]:
ceaps["DOCUMENTO"].value_counts()

In [ ]:
ceaps["COD_DOCUMENTO"]

In [ ]:
len(ceaps["COD_DOCUMENTO"].unique())

**Removendo colubas**

Dado que nem todas as colunas são necessárias, algumas serão removidas por otimização.

- `ANO`: todas as linhas têm o mesmo valor, 2021.
- `DOCUMENTO`: número atribuido pelo fornecedor, como número do voo ou da nota fiscal.*
- `COD_DOCUMENTO`: número de controle interno do Senado.*

*Deduzido, uma vez que o Senado Federak não respondeu aos questionamentos sobre o significado dessas colunas.

In [ ]:
ceaps.drop(columns=["ANO", "DOCUMENTO", "COD_DOCUMENTO"], inplace=True)

In [ ]:
ceaps.head()

In [ ]:
ceaps["TIPO_DESPESA"].unique()

In [ ]:
#Cannot convert string to float directly because of decimal separator
#Including decimal parameter in read_csv solves the problem

#ceaps["VALOR_REEMBOLSADO"] = ceaps["VALOR_REEMBOLSADO"].str.replace(",",".")
ceaps["VALOR_REEMBOLSADO"] = ceaps["VALOR_REEMBOLSADO"].astype("float")

In [ ]:
ceaps["VALOR_REEMBOLSADO"]

In [ ]:
ceaps["CNPJ_CPF"]

**Linhas em que o ano da coluna `DATA` não é 2021**

Para verificar quantas linhas têm o ano diferente de 2021 na coluna `DATA`, foi utilizado *list comprehension*. Em função da performance, tratar as datas como *strings* é uma opção melhor do que transformá-las para *datetime* e só depois extrair o ano. Isso pode ser verificado abaixo.

In [ ]:
#%timeit -n 1000 [x for x in ceaps['DATA'] if '2021' not in x] 

In [ ]:
#%timeit -n 1000 [x for x in ceaps["DATA"] if x.year != 2021]

In [ ]:
anos_errados = [x for x in ceaps["DATA"] if "2021" not in x]
index_anos_errados = [ceaps.index[ceaps["DATA"]==x][0] for x in anos_errados]

In [ ]:
print(f"Registros com ano diferente de 2021: {len(anos_errados)}")
print(f"Total de registros: {len(ceaps['DATA'])}")

print(f"{round(len(anos_errados)/len(ceaps['DATA'])*100, 2)}% dos registros com ano diferente de 2021")
    


**Uma vez que os índices das observações divergentes foram coletados para uso futuro, a coluna pode ser convertida para o formato *datetime*. Há uma observação cujo ano é 0202, provavelmente um erro de digitação. Isso, porém, impede que `pd.to_datetime` converta toda a coluna e a operação inteira falha. Isso pode ser solucionado utilizando `errors="coerce"`. As linhas que não puderem ser convertidas se tornam um nulo `NaT` (*not a time*).**

In [ ]:
ceaps["DATA"] = pd.to_datetime(ceaps["DATA"], format="%d/%m/%Y", errors="coerce")

**Ao verificar a existência de valores nulos na coluna DATA, apenas um registro é encontrado (aquele com ano 0202)**

In [ ]:
for x in ceaps["DATA"]:
    if pd.isnull(x) is True:
        print(x)

**Por se tratar de um sistema de reembolso, espera-se que haja alguma divergência entre o mês do registro e da despesa, uma vez que a despesa é feita antes do reembolso. No entanto, é importante avaliar o grau de divergência, especialmente em relação aos anos. Como pode ser observado, a maioria das despesas é de 2021, o ano fiscal dessa CEAPS. No entanto, há também despesas de 2022, 2019 e mesmo 2010 e 2002. É importante entender a natureza dessa divergência. São aventadas as seguinte hipóteses:**
- **Erro humano**: pode haver erro na inserção da data no sistema por parte do senador ou assessor responsável. Isso é especialmente  provável em registros como os mesmos dígitos em sequência distinta ou quando há vários registros com mesma data.
- **Fechamento do ano fiscal**: uma despesa que seja realizada durante o ano fiscal de 2020, mas cujo comprovante seja apresentado em 2021 pode ter seu lançamento em 2021, embora a despesa em si seja do ano anterior*;
- **Pagamento antecipado**: é possível que o período de prestação do serviço seja em 2021, ou se estenda a 2021, mas a contratação e pagamento tenha ocorrido antes. Nesse caso, possivelmente o valor será lançado na CEAPS seguinte. Isso é mais provável quando a data é próxima ao fechamento do ano fiscal*;
- **Nota fiscal emitida posteriormente**: é possível que o serviço tenha sido prestado duante o ano de 2021, mas a nota fiscal só tenha sido emitida posteriormente. Individualmente, isso pode ser comprovado em algumas notas fiscais em que o período de prestação do serviço é discriminado no documento. No entanto, nem todos os comprovantes trazem essa especificação;
- **Prática de ilícito**: é possível que essa divergência, ainda, seja um indicativo de ilícito relacionado ao uso CEAPS. Embora seja uma hipótese relativamente remota dada a facilidade de detecção, não pode ser totalmente descartada. Nesse caso, convém que haja uma investigação mais detida.

*Não houve resposta do Senado Federal acerca desses questionamentos, não sendo possível, portanto, afirmar com certeza.

In [ ]:
ano_desp = ceaps["DATA"].dt.year
mes_desp = ceaps["DATA"].dt.month

mes_reemb = ceaps["MES"]

In [ ]:
compara_ano = np.where(ano_desp == 2021, True, False)
cont = 0
for x in compara_ano:
    if x == False:
        cont += 1
print(f"Registros com ano diferente de 2021: {cont}")

In [ ]:
compara_mes = np.where(mes_desp == mes_reemb, True, False)
cont = 0
for x in compara_mes:
    if x == False:
        cont += 1
print(f"Registros com mês/data diferente do mês/reembolso: {cont}")

In [ ]:
ano_desp.value_counts()

In [ ]:
ceaps[ceaps["DATA"].dt.year == 2020]

**Convertendo o mês de número para nome**

A coluna `MES` contém os meses em que foram realizados os reembolsos. No *dataframe*, eles estão representados por inteiros. Ter os nomes dos meses é mais conveniente para a visualização e a análise dos dados. Nesse caso, optou-se pelo `map`para fazer a mudança. O módulo `calendar` do Python, combinado com `enumerate` pode ser usado para gerar um dicionário com os números como chaves e os nomes como valores. Eventualmente, o dicionário pode ser gerado com os nomes em inglês. Nesse caso, é preciso configurar o local/*locale*. Isso pode ser feito facilmente utilizando o módulo `locale`. Alternativamente, o dicionário pode ser construído manualmente com a mesma estrutura.

In [ ]:
#import locale
#locale.setlocale(locale.LC_ALL, 'pt_BR')
#d = dict(enumerate(calendar.month_name))

d = {1: 'Janeiro', 2: 'Fevereiro', 3: 'Março', 
     4: 'Abril', 5: 'Maio', 6: 'Junho', 
     7: 'Julho', 8: 'Agosto', 9: 'Setembro', 
     10: 'Outubro', 11: 'Novembro', 12: 'Dezembro'}

ceaps["MES"] = ceaps["MES"].map(d)

In [ ]:
ceaps.head()

***Dataframe* senadores**

A tabela `senadores`contém todos os senadores que exerceram mandato em 2021. Embora sejam 81 senadores, a tabela contém 91. Isso ocorre pelo fato de alguns serem suplentes, que exerceram o mandato em função de afastamento do titular. Desse modo, qualquer senador, seja titular ou suplente, que tenha utilizado a CEAPS aparece na tabela. Duas informações consideradas importantes não constam na tabela da CEAPS: UF e partido do senador, daí a necessidade dessa segunda tabela.

##### Colunas
- `SENADOR`: o nome do senador, padronizado segundo o nome da tabela da CEAPS;
- `PARTIDO`: o partido do senador conforme o nome no *site* do Senado, pode ser diferente entre titular e suplentes;
- `UF`: unidade federativa pela qual o senador exerce o mandato, necessariamente a mesma para titular e suplentes;
- `OBS`: observações em relação ao senador, como afastamento do mandato.

In [ ]:
senadores.head()

In [ ]:
senadores.info()

**Considerando o ano de 2022, cinco senadores não aparecem na em 2021. Dois deles são suplentes e assumiram as vagas do titular em 2022 (Alexandre Silveira e Fabio Garcia e três (Jorge Kajuru, Leila Barros e Reguffe) não utilizaram nenhum valor da CEAPS.**

In [ ]:
print("Senadores atuais que não constam na tabela da CEAPS")
for x in [x for x in senadores["SENADOR"]]:
    if x not in [x for x in ceaps["SENADOR"].unique()]:
        print(x)

In [ ]:
ceaps = ceaps.merge(senadores[["SENADOR","UF", "PARTIDO", "OBS"]], on="SENADOR", how="left")

# AED

### Maiores e menores gastos

Quando considerados os gastos por senador, observa-se que os maiores gastos tendem a se concentrar em estados do Norte do país. Quanto mais distante o estado de Brasília, maior o valor da cota a que o senador tem direito.

In [ ]:
ceaps.groupby(["SENADOR", "UF"]).sum().sort_values(by="VALOR_REEMBOLSADO",ascending=False).head(10)

Por outro lado, o inverso não permite a mesma observação diretamente. Nesse caso é preciso considerar o tempo de mandato que o senador exerceu em 2021. Senadores afastados e suplentes em exercício terão menores valores. Nesse caso, convém considerar cada senador individualmente e as observações relativas a ele.

In [ ]:
ceaps.groupby(["SENADOR", "UF", "OBS"]).sum().sort_values(by="VALOR_REEMBOLSADO",ascending=False).tail(11)

### Gastos por partido

- Considerando a possibilidade de troca de partido por parte dos senadores durante seu mandato, pode haver algumas distorções em função desse fato. 
- Dessa forma, a fim de estabelecer um critério para atribuir o partido de cada senador, foi considerado o tempo de mandato exercido em 2021. 
- Assim, **um senador que tenha mudado de partido estará no partido a que esteve filiado por mais tempo em 2021**. Esse dado não consta no *site* do Senado e foi necessário recorrer à imprensa. 
- É preciso considerar o número de senadores por partido, uma vez que partidos com mais parlamentares tendem a ter gastos maiores.
- A cota é de responsabilidade de cada parlamentar, a informação do seu partido tem natureza complementar.

In [ ]:
ceaps.groupby(["PARTIDO"]).sum().sort_values(by="VALOR_REEMBOLSADO",ascending=False)

In [ ]:
senadores["PARTIDO"].value_counts()

### Reembolsos a pessoas físicas

Um valor significativo (cerca de R$ 1,2 mi) foi destinado a reembolsos a pessoas físicas. Esse é um dado importante a ser considerado, uma vez que a disponibilidade de informações públicas sobre pessoas físicas é menor do que das empresas. É, portanto, um ponto de atenção do ponto de vista da transparência. Nesse sentido, o contribuinte pode  questionar diretamente o parlamentar a respeito, por exemplo, sobre os motivos que levaram à opção por pessoa física e não jurídica, valor pago pelo bem/serviço e relação entre o parlamentar ou seus assessores com o prestador.

In [ ]:
ceaps[ceaps["CNPJ_CPF"].apply(len)==14]["VALOR_REEMBOLSADO"].sum()

In [ ]:
ceaps[ceaps["CNPJ_CPF"].apply(len)==14].groupby(["SENADOR", "FORNECEDOR"]).sum().sort_values("VALOR_REEMBOLSADO", ascending=False)
#ceaps[len(ceaps["CNPJ_CPF"]==14)]

In [ ]:
ceaps2 = ceaps

### Principais empresas fornecedoras

Embora um valor importante seja destinado a pessoas físicas, as empresas respondem pela maioria dos reembolsos, tanto em número quanto de valores recebidos.

In [ ]:
ceaps["CNPJ_CPF"].apply(len).value_counts()

A principal dificuldade encontrada nesse ponto é a padronização dos nomes dos fornecedores. Uma vez que eles são inseridos manualmente, as diferenças mínimas podem levar a resultados incorretos ao agrupar pelo valor recebido. Isso fica evidente com o CNPJ 16.978.175/0001-08. A empresa em questão é a maior recebedora de valores. No entanto, quando o agrupamento é feito pelo nome, esse valor se divide em diferentes linhas com grafias diferentes. Embora o simples agrupamento pelo CNPJ solucionasse esse problema, esse número por si traz poucas informações sobre a empresa.

In [ ]:
ceaps.groupby(["CNPJ_CPF"]).sum().sort_values(by="VALOR_REEMBOLSADO",ascending=False).head(10)

In [ ]:
ceaps.groupby(["CNPJ_CPF", "FORNECEDOR"]).sum().sort_values(by="VALOR_REEMBOLSADO",ascending=False).head(10)

Em função disso, havia a necessidade de padronização dos nomes das empresas. Isso poderia ser feito pelo *site* da Receita Federal, responsável pelo registro dos CNPJs. Para isso, as seguintes opções existiam:
1. **Realizar uma busca manual de cada CNPJ**: essa opção se torna inviável pelo número de registros. Existem pouco menos de 16.500 CNPJs registrados na tabela. Mesmo quando reduzido aos valores únicos, esse número se aproxima de 2800 CNPJs. 
2. **Utilizar *web* scraping**: o *site* da Receita Federal permite consultas a qualquer CNPJ, no entanto o uso de *web scraping* se torna essencialmente inviável pelo uso de CAPTCHA.
3. **API da Receita Federal**: há uma API que permite consultas de dados pelo número do CNPJ, sua utilização, porém, é paga por consulta realizada. Dado o preço e o volume de consultas, essa opção se torna proibitiva no escopo desse projeto.
4. **Utilizar a base completa de CNPJs**: mensalmente, a Receita Federal disponibiliza a base completa do CNPJ. Esse dados podem ser baixados e consultados livremente. O principal obstáculo aqui é o grande volume de dados, uma vez que se trata da base completa, com todos os tipos de informações registradas.
5. **Fazer uso de uma API não-oficial**: dada essa ser uma necessidade que existe em outras aplicações, existem diversos projetos de código aberto para lidar com as informações desse tipo. Nesse caso, os dados que são disponibilizados pela Receita Federal, são processados e ficam acessíveis por meio de consultas. A grande vantagem desse método é diminuição da carga de trabalho e processamento, uma vez que são consultados apenas os dados relevantes e necessários para a aplicação.

Esse último foi o caminho escolhido para esse projeto. No entanto, em caso de aplicações que exijam um altíssimo grau de atualização e integridade das informações, o melhor caminho talvez seja o uso da API oficial da Receita Federal. Porém, no escopo do projeto, foi considerado que o caminho escolhido é suficientemente confiável para o objetivo de padronização de nomes, bem como de outras informações julgadas relevantes.

Foi escolhido o Minha Receita (https://github.com/cuducos/minha-receita) como API a ser utilizada. Esse projeto utiliza os dados públicos da Receita Federal para facilitar a consulta às informações de CNPJs. 

A API funciona por meio de requisições HTTP do tipo `GET` retornando um JSON com todos os dados do CNPJ consultado. Para o envio das requisições, foi o utilizado o módulo `requests`.

Inicialmente, foram escolhidos os campos considerados relevantes para cada empresa, que foram os seguintes:
- CNPJ: o número de registro
- UF e município
- Razão social e nome fantasia: campos a serem usados para identificar cada empresa mais facilmente
- Porte: descreve o tamanho da empresa
- CNAE: Classificação Nacional de Atividades Econômicas descreve a principal atividade da empresa e pode ser usado para detecção de possíveis ilícitos, como empresas recebendo por serviços estranhos às suas atividades.
- Início das atividades: verificação de possíveis ilícitos, como abertura empresas de fachada abertas muito recentemente.

Uma vez extraídos os dados, foram colocados em um *DataFrame* e salvos em CSV para uso posterior sem a necessidade de novas consultas.

In [ ]:
"""
cnpjs = [x for x in ceaps["CNPJ_CPF"].unique() if len(x)==18]

dados_cnpj = ["cnpj", "uf", "municipio", "razao_social", "nome_fantasia", 
              "porte", "cnae_fiscal_descricao", "data_inicio_atividade"]

empresas = pd.DataFrame(columns=dados_cnpj)

cont = 0

for x in cnpjs:
    request = requests.get("https://minhareceita.org/"+x)
    json = request.json()
    nova_linha = [json.get(x) for x in dados_cnpj]
    empresas.loc[len(empresas)] = nova_linha
    cont += 1
    if cont%100==0 or cont==len(cnpjs):
        print(f"{cont} CNPJs processados")
        
"""

In [ ]:
#empresas.to_csv("dados_empresas_ceaps_2021.csv", index=False)

In [ ]:
ceaps["CNPJ_CPF_NUM"] = ceaps["CNPJ_CPF"]
ceaps["CNPJ_CPF_NUM"] = ceaps["CNPJ_CPF_NUM"].str.replace("/","", regex=True)
ceaps["CNPJ_CPF_NUM"] = ceaps["CNPJ_CPF_NUM"].str.replace("-","", regex=True)
ceaps["CNPJ_CPF_NUM"] = ceaps["CNPJ_CPF_NUM"].str.replace(".","", regex=True)

In [ ]:
reembolsos = ceaps.groupby(by="CNPJ_CPF_NUM").sum()
reembolsos.reset_index(inplace=True)

reembolsos["CNPJ_CPF_NUM"] = reembolsos["CNPJ_CPF_NUM"].astype("int64")

In [ ]:
#Left join because reembolsos has more rows (they include CPFs)
empresas_reembolsos = pd.merge(empresas, reembolsos, left_on="cnpj", right_on="CNPJ_CPF_NUM", how="left")

In [ ]:
empresas_reembolsos.rename(columns={"VALOR_REEMBOLSADO":"valor_recebido"}, inplace=True)
empresas_reembolsos.drop(columns="CNPJ_CPF_NUM", inplace=True)

empresas_reembolsos["valor_recebido"] = empresas_reembolsos["valor_recebido"].astype("float")
empresas_reembolsos["cnpj"] = empresas_reembolsos["cnpj"].astype("str")

empresas_reembolsos["data_inicio_atividade"] = pd.to_datetime(empresas_reembolsos["data_inicio_atividade"], format="%Y-%m-%d", errors="coerce")

In [ ]:
empresas_reembolsos.sort_values(by="valor_recebido", ascending=False)[:5]

In [ ]:
empresas_reembolsos.groupby(by="porte").sum()

In [ ]:
empresas_reembolsos.groupby(by="uf").sum().sort_values(by="valor_recebido", ascending=False)

In [ ]:
reembolsos_cnae = empresas_reembolsos.groupby("cnae_fiscal_descricao").sum()

In [ ]:
reembolsos_cnae["percentual"] = reembolsos_cnae["valor_recebido"]/empresas_reembolsos["valor_recebido"].sum()*100

reembolsos_cnae.sort_values(by="percentual", ascending=False, inplace=True)

reembolsos_cnae["acumulado"] = reembolsos_cnae["percentual"].cumsum()

In [ ]:
#reembolsos_cnae.head(10)

In [ ]:
ceaps.groupby(["PARTIDO"]).sum().sort_values(by="VALOR_REEMBOLSADO",ascending=False)

In [ ]:
totais_por_partido = ceaps.groupby(["PARTIDO"]).sum()
senadores_por_partido = senadores["PARTIDO"].value_counts()

In [ ]:
totais_partido_senadores = totais_por_partido.merge(senadores_por_partido, left_index=True, right_index=True)
totais_partido_senadores.rename(columns={"PARTIDO":"SENADORES"}, inplace=True)

totais_partido_senadores = totais_partido_senadores.assign(NUM_REEMBOLSOS=ceaps["PARTIDO"].value_counts())

totais_partido_senadores["MEDIA_POR_PARTIDO"] = totais_partido_senadores["VALOR_REEMBOLSADO"]/totais_partido_senadores["SENADORES"]
totais_partido_senadores["MEDIA_POR_PARTIDO"] = totais_partido_senadores["MEDIA_POR_PARTIDO"].round(2)

totais_partido_senadores["MEDIA_REEMBOLSO"] = totais_partido_senadores["VALOR_REEMBOLSADO"]/totais_partido_senadores["NUM_REEMBOLSOS"]
totais_partido_senadores["MEDIA_REEMBOLSO"] = totais_partido_senadores["MEDIA_REEMBOLSO"].round(2)

totais_partido_senadores.reset_index(inplace=True)
totais_partido_senadores.rename(columns={"index":"PARTIDO"}, inplace=True)

In [ ]:
#totais_partido_senadores

In [ ]:
ceaps.groupby(["PARTIDO", "SENADOR"]).sum()

In [ ]:
ceaps.groupby(["UF", "SENADOR"]).sum()


In [ ]:
#ceaps["UF"].unique()

### Atribuindo estados às regiões

Um aspecto importante é a categorização por região brasileira. Ao mesmo tempo em que os senadores de estados mais distantes de Brasília têm direito a valores maiores da CEAPS, em muitos desses estados existe uma carência de serviços públicos, além de muitos, particularmente no Norte e Nordeste do país terem indicadores de desenvolvimento humano abaixo do ideal. Dessa forma, essa é uma métrica importante de ser considerada na avaliação do destino do dinheiro do contribuinte. Muito embora os senadores representem suas unidades federativas, em última instância detém um mandato popular e é à população a quem devem responder. Assim, é necessário avaliar, junto com outros dados, o retorno em termos concretos do gasto realizado pelos parlamentares. Em outras palavras, quanto desse dinheiro que é usado livremente por senadores se reverte em benefícios concretos para a população.

Aqui foi utilizado um dicionário com as siglas das unidades federativas como chave e a região como valor. Em seguida, convertido em série e *DataFrame* e incorporado à tabela principal.

In [ ]:
regioes = {
    'AC': 'Norte', 'AL': 'Nordeste', 'AM': 'Norte', 
    'AP': 'Norte', 'BA': 'Nordeste', 'CE': 'Nordeste', 
    'DF': 'Centro-Oeste', 'ES': 'Sudeste', 'GO': 'Centro-Oeste', 
    'MA': 'Nordeste', 'MG': 'Sudeste', 'MS': 'Centro-Oeste', 
    'MT': 'Centro-Oeste', 'PA': 'Norte', 'PB': 'Nordeste', 
    'PE': 'Nordeste', 'PI': 'Nordeste', 'PR': 'Sul', 
    'RJ': 'Sudeste', 'RN': 'Nordeste', 'RO': 'Norte', 
    'RR': 'Norte', 'RS': 'Sul', 'SC': 'Sul', 
    'SE': 'Nordeste', 'SP': 'Sudeste', 'TO': 'Norte'
}
regioes_Ser = pd.Series(regioes)

In [ ]:
ceaps = ceaps.merge(regioes_Ser.to_frame(), right_index=True, left_on="UF", how="left")
ceaps.rename(columns={0:"REGIAO"}, inplace=True)

In [ ]:
empresas["cnpj"] = empresas["cnpj"].astype("int64")
ceaps["CNPJ_CPF_NUM"] = ceaps["CNPJ_CPF_NUM"].astype("int64")

ceaps = ceaps.merge(empresas, left_on="CNPJ_CPF_NUM", right_on="cnpj", how="left")

In [ ]:
#ceaps.head()

In [ ]:
#ceaps.info()

In [ ]:
ceaps["cnpj"] = ceaps["cnpj"].astype("str")
ceaps["CNPJ_CPF_NUM"] = ceaps["CNPJ_CPF_NUM"].astype("str")

In [ ]:
ceaps.head()

In [ ]:
ceaps["razao_social"].fillna(ceaps["FORNECEDOR"], inplace=True)
ceaps["porte"].fillna("Pessoa física", inplace=True)
ceaps["cnae_fiscal_descricao"].fillna("Pessoa física", inplace=True)

In [ ]:
#ceaps.head()

### Simplicando o tipo de despesa

Os nomes dos diferentes tipos de despesas da CEAPS são muito longos. Para fins de análise convém que sejam reduzidos para termos mais sucintos, mais ainda significativos. Para isso, foi utilizado um dicionário com os nomes originais e suas reduções e criada uma nova coluna `CAT_DESP`.

In [ ]:
dict_desp = {
        'Aluguel de imóveis para escritório político, compreendendo despesas concernentes a eles.':"Locação/manutenção de escritório político",
        'Aquisição de material de consumo para uso no escritório político, inclusive aquisição ou locação de software, despesas postais, aquisição de publicações, locação de móveis e de equipamentos. ':"Despesas de escritório político",
        'Contratação de consultorias, assessorias, pesquisas, trabalhos técnicos e outros serviços de apoio ao exercício do mandato parlamentar': "Serviços de apoio",
        'Divulgação da atividade parlamentar':"Divulgação",
        'Locomoção, hospedagem, alimentação, combustíveis e lubrificantes': "Locomoção, hospedagem e alimentação",
        'Serviços de Segurança Privada': "Segurança privada",
        'Passagens aéreas, aquáticas e terrestres nacionais': "Passagens"   
}

desp_Ser = pd.Series(dict_desp)

In [ ]:
uf_sen_cat = ceaps.groupby(["REGIAO", "UF", "SENADOR", "MES",  "TIPO_DESPESA", "razao_social", "cnae_fiscal_descricao"]).sum()
uf_sen_cat.reset_index(inplace=True)

uf_sen_cat = uf_sen_cat.merge(desp_Ser.to_frame(), left_on="TIPO_DESPESA", right_index=True, how="left")
uf_sen_cat.rename(columns={0:"CAT_DESP"}, inplace=True)

uf_sen_cat.columns = [x.upper() for x in uf_sen_cat.columns]

uf_sen_cat

In [ ]:
dd = dict(ceaps["TIPO_DESPESA"].value_counts(ascending=False))
cat_despesas = ["Locação/manutenção de escritório político", "Divulgação parlamentar", "Passagens", 
                "Locomoção, alimentação e hospedagem", "Consultorias e assessorias", 
                "Material de consumo esc. político", "Segurança privada"]

In [ ]:
ax = sns.countplot(data=ceaps, x="TIPO_DESPESA", order=dd)

ax.set(title="Reembolsos por categoria da CEAPS em 2021", xlabel="Categoria", ylabel="Ocorrências", xticklabels="")
ax.legend(ax.containers[0], cat_despesas, bbox_to_anchor=(1,1), title="Categorias")
ax.bar_label(ax.containers[0], padding=3)
ax.set_ylim(0, 6500)

# For future reference
# ax.containers[0] returns the objects being plotted in the graph,
# which, in this case, are the bar of the categories.
# As such, it can be passed as handler (that seems to be a reference)
# for other objects/functions, the legend in this case

In [ ]:
pos = ["top center" if x not in ["REDE", "CIDADANIA"] else "bottom center" for x in totais_partido_senadores["PARTIDO"]]
fig = px.scatter(data_frame=totais_partido_senadores, x="MEDIA_POR_PARTIDO", y="SENADORES", 
                 text="PARTIDO", title="Gasto médio por partido pelo número de senadores")


hovertemp = ["<b>Partido</b>: %{text}<br>",
             "<b>Senadores</b>: %{y}<br>",
             "<b>Gasto médio por senador</b>: R$ %{x:,}<br>"]
hovertemp = "".join(hovertemp)

fig.update_traces(textposition=pos, hovertemplate=hovertemp)
fig.update_layout(separators=",.", xaxis_title="Média de reembolso por senador", yaxis_title="Senadores por partido")

In [ ]:
ano_inicio_grouped = empresas_reembolsos.groupby(empresas_reembolsos["data_inicio_atividade"].dt.year).sum().sort_values(by="valor_recebido", ascending=False)
ano_inicio_grouped.reset_index(inplace=True)

px.histogram(data_frame=ano_inicio_grouped, x="data_inicio_atividade", y="valor_recebido")

In [ ]:
porte_grouped = empresas_reembolsos.groupby(empresas_reembolsos["porte"]).sum().sort_values(by="valor_recebido", ascending=False)
porte_grouped.reset_index(inplace=True)


px.bar(data_frame=porte_grouped, x="porte", y="valor_recebido")

In [ ]:
nordeste = uf_sen_cat[uf_sen_cat["REGIAO"]=="Nordeste"]
norte = uf_sen_cat[uf_sen_cat["REGIAO"]=="Norte"]
sudeste = uf_sen_cat[uf_sen_cat["REGIAO"]=="Sudeste"]
sul = uf_sen_cat[uf_sen_cat["REGIAO"]=="Sul"]
centro_oeste = uf_sen_cat[uf_sen_cat["REGIAO"]=="Centro-Oeste"]

In [ ]:
def customwrap(s,width=60):
    return "<br>".join(textwrap.wrap(s,width=width))

In [ ]:
path=["REGIAO", "UF", "SENADOR", "MES", "CAT_DESP", "RAZAO_SOCIAL", 
      nordeste["CNAE_FISCAL_DESCRICAO"].apply(customwrap)]

fig_temp = px.treemap(data_frame=nordeste, path=path, values="VALOR_REEMBOLSADO", template="presentation", 
                      color_discrete_sequence=px.colors.qualitative.D3)

hovertemp = ["<b>Valor</b>: R$ %{value:,}<br>"]
hovertemp = "".join(hovertemp)

fig_temp.update_traces(textposition="middle center", hovertemplate=hovertemp, 
                       textinfo = "label", texttemplate="<b>CNAE</b>: %{label}<br> <b>Valor pago no mês</b>: R$ %{value}")
fig_temp.update_layout(separators=",.")

In [ ]:
totais_partido = ceaps.groupby(["PARTIDO"]).sum()
totais_partido.reset_index(inplace=True)

In [ ]:
fig_partidos = px.bar(totais_partido[["PARTIDO","VALOR_REEMBOLSADO"]], x="PARTIDO", 
                      y="VALOR_REEMBOLSADO", color="VALOR_REEMBOLSADO",  color_continuous_scale=px.colors.sequential.YlOrRd,
                     title="Gastos da CEAPS por partido do senador")

hovertemp = ["<b>Partido</b>: %{x}<br>","<b>Gasto total do partido</b>: R$ %{y:,}"]
hovertemp = "".join(hovertemp)    

fig_partidos.update_traces(hovertemplate=hovertemp)
fig_partidos.update_layout(separators=",.", yaxis_title="Gasto total", xaxis_title="Partidos", 
                           xaxis={"categoryorder": 'total descending'})

In [ ]:
#Boxplot partido
#Senadores que mais e menos gastaram

In [ ]:
sen_part_tot = ceaps.groupby(["PARTIDO","SENADOR"]).sum()
sen_part_tot.reset_index(inplace=True)

fig6 = px.treemap(data_frame=sen_part_tot,
            path=["PARTIDO","SENADOR"], values="VALOR_REEMBOLSADO")

fig6.update_traces(hovertemplate="<b>Valor gasto</b>: R$ %{value:,}", textposition="middle center", textinfo = "label",
                   texttemplate="<b>Senador</b>: %{label}<br><b>Valor gasto no ano</b>: R$ %{value}")

fig6.update_layout(separators=",.")

In [ ]:
estados = ceaps.groupby(["UF"]).sum()
estados.reset_index(inplace=True)

In [ ]:
#estados.to_csv("estados_total.csv", decimal=",")

In [ ]:
from urllib.request import urlopen
with urlopen("https://raw.githubusercontent.com/codeforamerica/click_that_hood/master/public/data/brazil-states.geojson") as response:
    estados_geo = json.load(response)

In [ ]:
estados_geo = json.load(open("./brasil_estados.json"))

mapa = px.choropleth(estados, geojson=estados_geo, locations="UF", width=1000, height=1000,
                     color="VALOR_REEMBOLSADO", projection="mercator", color_continuous_scale=px.colors.sequential.YlOrRd)

hovertemp = ["<b>UF</b>: %{location}", "<b>Gasto total</b>: R$ %{z:,}"]
hovertemp = "<br>".join(hovertemp)    

mapa.update_traces(hovertemplate=hovertemp)

mapa.update_geos(fitbounds="geojson", visible=False)

mapa.update_coloraxes(colorbar_len=0.4, colorbar_title_text="Valor gasto<br>pelos senadores", colorbar_thickness=15)

mapa.update_layout(separators=",.", margin={"r":0,"t":0,"l":0,"b":0}, hoverlabel=dict(font_size=18))    

In [ ]:
mapa.write_html("file.html")

In [ ]:
import folium

In [ ]:
m = folium.Map(location=[-10, -55], zoom_start=4)

In [ ]:
folium.Choropleth(
    geo_data=estados_geo,
    name='choropleth',
    data=estados,
    columns=['UF', 'VALOR_REEMBOLSADO'],
    key_on='feature.id',
    fill_color='YlOrRd',
    fill_opacity=1,
    line_opacity=0.2,
    legend_name='Valor gasto (R$)',
    popup='<b>Stackoverflow</b><br><br>2021.01.01'
).add_to(m)

In [ ]:
m